In [3]:
import sys
import numpy as np
import matplotlib.pyplot as plt
import cv2
from pathlib import Path

folder = 'fig'

## 평균이동 Mean shift

In [13]:
# cap = cv2.VideoCapture(Path(folder, 'Billard.mp4'))
cap = cv2.VideoCapture(Path(folder, 'vtest.avi'))

if not cap.isOpened():
    print('Video open failed!')
    sys.exit()

ret, frame = cap.read()

if not ret:
    print('frame read failed!')
    sys.exit()

(x, y, w, h) = cv2.selectROI("ROI", frame)
rc = (x, y, w, h)

roi = frame[y:y+h, x:x+w] # BGR
roi_hsv = cv2.cvtColor(roi, cv2.COLOR_BGR2HSV) # BGR -> HSV, H:색상, S:채도, V:밝기

## 색상의 히스토그램
channels = [0, 1]
ranges = [0, 180, 0, 256]
hist = cv2.calcHist(roi_hsv, channels, None, [90, 128], ranges)

## Mean shift 종료 기준
term_crit = (cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 1)

while True:
    ret, frame = cap.read()

    if not ret:
        break

    frame_hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    backproj = cv2.calcBackProject([frame_hsv], channels, hist, ranges, 1)

    ## Mean shift
    _, rc = cv2.meanShift(backproj, rc, term_crit)

    cv2.rectangle(frame, rc, (0, 0, 255), 2)

    cv2.imshow('frame', frame)

    if cv2.waitKey(20) == 27:
        break

cap.release()
cv2.destroyAllWindows()

### 캠시프트(Camshift)

In [9]:
# cap = cv2.VideoCapture(Path(folder, 'Billard.mp4'))
cap = cv2.VideoCapture(Path(folder, 'vtest.avi'))

if not cap.isOpened():
    print('Video open failed!')
    sys.exit()

ret, frame = cap.read()
if not ret:
    print('frame read failed!')
    sys.exit()

(x, y, w, h) = cv2.selectROI("ROI", frame)
rc = (x, y, w, h)

roi = frame[y:y+h, x:x+w]
roi_hsv = cv2.cvtColor(roi, cv2.COLOR_BGR2HSV)

## 히스토그램
channels = [0, 1] # h, s
ranges = [0, 180, 0 , 255]
hist = cv2.calcHist([roi_hsv], channels, None, [90, 128], ranges) # 기준 영상의 히스토그램

## camshift 종료 조건
term_crit = (cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 1)

while True:
    ret, frame = cap.read()

    if not ret:
        break

    frame_hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    backproj = cv2.calcBackProject([frame_hsv], channels, hist, ranges, 1)

    ## Camshift
    ret, rc = cv2.CamShift(backproj, rc, term_crit)

    cv2.rectangle(frame, rc, (0, 0, 255), 2)
    cv2.ellipse(frame, ret, (0, 255, 0), 2)
    cv2.imshow('frame', frame)

    if cv2.waitKey(20) == 27:
        break

cap.release()
cv2.destroyAllWindows()

### 모션 벡터 (Motion vector)

### Lucas-Kanade optical flow

In [61]:
src1 = cv2.imread(Path(folder, "frame1.jpg"))
src2 = cv2.imread(Path(folder, "frame2.jpg"))

if src1 is None or src2 is None:
    print('Image load failed!')
    sys.exit()

gray1 = cv2.cvtColor(src1, cv2.COLOR_BGR2GRAY)
pt1 = cv2.goodFeaturesToTrack(gray1, 50, 0.01, 10)
pt2, status, err = cv2.calcOpticalFlowPyrLK(src1, src2, pt1, None)

dst = cv2.addWeighted(src1, 0.5, src2, 0.5, 0.0)

for i in range(pt2.shape[0]):
    if status[i, 0] == 0:
        continue

    cv2.circle(dst, (int(pt1[i, 0, 0]), int(pt1[i, 0, 1])), 4, (0, 255, 255), 2, cv2.LINE_AA)
    cv2.circle(dst, (int(pt2[i, 0, 0]), int(pt2[i, 0, 1])), 4, (0, 0, 255), 2, cv2.LINE_AA)
    cv2.arrowedLine(dst, (int(pt1[i, 0, 0]), int(pt1[i, 0, 1])), (int(pt2[i, 0, 0]), int(pt2[i, 0, 1])), (0, 255, 0), 2)

cv2.imshow('dst', dst)
cv2.waitKey()
cv2.destroyAllWindows()



# for i in pt1:
#     (x, y) = i[0]
#     cv2.circle(src1, (int(x), int(y)), 4, (0, 0, 255), 2, cv2.LINE_AA)

# cv2.imshow('src1', src1)
# cv2.waitKey()
# cv2.destroyAllWindows()

# src1_rgb = cv2.cvtColor(src1, cv2.COLOR_BGR2RGB)
# src2_rgb = cv2.cvtColor(src2, cv2.COLOR_BGR2RGB)

# plt.figure(figsize=(12, 6))
# plt.subplot(121), plt.imshow(src1_rgb), plt.title('frame1'), plt.axis('off')
# plt.subplot(122), plt.imshow(src2_rgb), plt.title('frame2'), plt.axis('off')
# plt.show()

### Dense optical flow

In [63]:
def draw_flow(img, flow, step=16):
    h, w = img.shape[:2]
    y, x = np.mgrid[step/2:h:step, step/2:w:step].reshape(2, -1).astype(int)
    fx, fy = flow[y, x].T
    lines = np.vstack([x, y, x+fx, y+fy]).T.reshape(-1, 2, 2)
    lines = np.int32(lines + 0.5)
    vis = cv2.cvtColor(img, cv2.COLOR_GRAY2BGR)
    cv2.polylines(vis, lines, 0, (0, 255, 255), lineType=cv2.LINE_AA)

    for (x1, y1), (_x2, _y2) in lines:
        cv2.circle(vis, (x1, y1), 1, (0, 128, 255), -1, lineType=cv2.LINE_AA)

    return vis


cap = cv2.VideoCapture(Path(folder, 'vtest.avi'))

if not cap.isOpened():
    print("Video open failed")
    sys.exit()

ret, frame1 = cap.read()
gray1 = cv2.cvtColor(frame1, cv2.COLOR_BGR2GRAY)

while True:
    ret, frame2 = cap.read()

    if not ret:
        break

    gray2 = cv2.cvtColor(frame2, cv2.COLOR_BGR2GRAY)
    flow = cv2.calcOpticalFlowFarneback(gray1, gray2, None, 0.5, 3, 13, 3, 5, 1.1, 0)

    cv2.imshow('frame2', draw_flow(gray2, flow))

    if cv2.waitKey(20) == 27:
        break

    gray1 = gray2

cap.release()
cv2.destroyAllWindows()